# Residue Hyperdimensional Computing

In this notebook we implement Kymn et al. (2023), 
[*Computing with Residue Numbers in High-Dimensional Representation*](https://arxiv.org/abs/2311.04872).

In [71]:
import numpy as np
from vsa.vocabulary import Vocabulary
from typing import Dict, List, Tuple, Literal
import math
import cmath

In [274]:
class RHC(Vocabulary):
    """
    Residue Hyperdimensional Computing.

    Attributes:
        dim: An integer representing the dimensionality of the vectors produced.
        symbols: An optional dictionary mapping labels to symbols.
        moduli: A list of integers which are the moduli of the encoding scheme.
    """

    _dim: int
    _symbols: Dict[str, np.ndarray]
    _moduli: List[int]
    _roots: Dict[int, np.ndarray]
    _phis: Dict[int, np.ndarray]

    # dictionary mapping each moduli m_i to an m_i x self.dim matrix,
    # used for residue decoding
    _codebook: Dict[int, np.ndarray]

    def __init__(
        self, dim: int, moduli: List[int], symbols: Dict[str, np.ndarray] = {}
    ) -> None:
        self._dim = dim
        self._symbols = symbols
        self._moduli = moduli
        self.__post_init__()

    def __post_init__(self) -> None:
        self._roots, self._phis = self._get_phis()

        for key, value in self._phis.items():
            self._symbols[str(key)] = value

        self._invs = self._get_invs()

        self._codebook = {}
        for modulus in self.moduli:
            modbook = np.zeros((modulus, self.dim), dtype=complex)
            for i in range(modulus):
                modbook[i, :] = self._phis[modulus] ** (i + 1)

            self._codebook[modulus] = modbook.T

    @property
    def dim(self) -> int:
        """
        The dimensionality of the RHC.
        """
        return self._dim

    @property
    def symbols(self) -> Dict[str, np.ndarray]:
        """
        The set of symbols and their values.
        """
        return self._symbols

    @property
    def moduli(self) -> List[int]:
        """
        The moduli of the RHC.
        """
        return self._moduli

    def _get_phis(self) -> Tuple[Dict[int, np.ndarray], Dict[int, np.ndarray]]:
        """
        For each modulus `m`, sample from the `m`-th roots of unity to
        produce an `self.dim`-dimensional vector.

        Returns:
            A tuple of a dictionary mapping moduli to their real roots of unity,
            and a dictionary mapping moduli to the complex angle roots of unity.
        """

        real_phis = {}
        phis = {}
        for modulus in self.moduli:
            real_phis[modulus] = self._roots_of_unity(modulus)
            phis[modulus] = np.exp(cmath.sqrt(-1) * real_phis[modulus])
        return real_phis, phis

    def _roots_of_unity(self, modulus: int) -> np.ndarray:
        """Sample from the `modulus` roots of unity to create a
            `self.dim`-dimensional vector.

        Args:
            modulus: An integer `m` to sample from the `m`th roots of unity.

        Returns:
            A real vector of sampled elements from the `modulus` roots of unity.
        """

        # generate the `modulus`-roots of unity of the unit circle
        roots = [2 * math.pi]
        incr = (2 * math.pi) / modulus
        curr = incr
        while curr < 2 * math.pi:
            roots.append(curr)
            curr += incr

        # create a vector of angles sampled from the `modulus`-roots of unity
        sample_roots = np.vectorize(lambda _: np.random.choice(roots))
        angles = sample_roots(np.zeros(self.dim))
        return angles

    def _get_invs(self) -> Dict[int, np.ndarray]:
        """
        Anti-base vectors for each moduli defined by the modular
        multiplicative inverses of the real angles.

        Returns:
            A dictionary with the moduli as the keys and the inverses
            of their associated vectors as values.
        """
        # Implementation is ripped from `inverse_phases` function by Kymn
        invs = {}
        for modulus, roots in self._roots.items():
            inv = np.zeros_like(roots)
            for i in range(roots.size):
                if np.round(np.angle(roots[i])).astype(int) == 0:
                    inv[i] = 0
                else:
                    spin = int(np.round(np.angle(roots[i]) * modulus))
                    inv[i] = pow(int(np.round(roots[i])), -1, modulus)
            invs[modulus] = inv
        return invs

    def bind(self, x: np.ndarray, y: np.ndarray) -> np.ndarray:
        """
        Bind two `RHC` vectors to create a new, orthogonal vector.

        Returns:
            A new `self.dim`-dimensional vector.
        """
        return np.multiply(x, y)

    def superpose(self, x: np.ndarray, y: np.ndarray) -> np.ndarray:
        """
        Put vectors `x` and `y` into superposition.

        Returns:
            A new `self.dim`-dimensional vector.
        """
        return x + y

    def inv(self, x: np.ndarray) -> np.ndarray:
        """
        Invert a `self.dim`-dimensional vector.

        Returns:
            A new `self.dim`-dimensional vector which is the conjugate of `x`.
        """
        return np.conjugate(x)

    def add(self, x: np.ndarray, y: np.ndarray) -> np.ndarray:
        """
        Add two encoded integers, or: `x (n_1) * x (n_2) = x (n_1 + n_2)`

        Returns:
            A new complex `np.ndarray` which satisfies the above property.
        """
        return self.bind(x, y)

    def _resonator_mul(self, x: np.ndarray, y: np.ndarray) -> np.ndarray:
        """
        Implement multiplicative binding by decoding `y` into an integer,
        and then performing element-wise exponentiation.

        Returns:
            A new complex `np.ndarray`.
        """
        y_n = self.decode(y)
        return np.pow(x, y)

    def _kymn_mul(self, x: np.ndarray, y: np.ndarray) -> np.ndarray:
        """
        Implement Kymn's method for multiplicative binding, which requires
        calling a resonator network to recover the base vectors of `x`
        and `y`, and then using the anti-base vector for each to
        perform multiplicative binding.

        Returns:
            A new complex `np.ndarray`.
        """
        raise NotImplementedError("TODO")

    def mul(
        self,
        x: np.ndarray,
        y: np.ndarray,
        decoder_method: Literal["decode", "kymn"] = "decode",
    ) -> np.ndarray:
        if decoder_method == "decode":
            return self._resonator_mul(x, y)
        elif decoder_method == "kymn":
            return self._kymn_mul(x, y)
        else:
            raise ValueError("Unexpected decoder variant", decoder_method)

    def sub(self, x: np.ndarray, y: np.ndarray) -> np.ndarray:
        """
        Subtract two encoded integers.
        """
        return self.add(x, self.inv(y))

    def div(
        self,
        x: np.ndarray,
        y: np.ndarray,
        decoder_method: Literal["decode", "kymn"] = "decode",
    ) -> np.ndarray:
        raise NotImplementedError("TODO")

    def sim(self, x: np.ndarray, y: np.ndarray) -> float:
        """
        Args:
            x: A `self.dim`-dimensional vector.
            y: A `self.dim`-dimensional vector.

        Returns:
            A float in `[-1, 1]` measuring the similarity between the two
                vectors. `0` if orthogonal, `-1, 1` if related.
        """

        return np.dot(x, np.conjugate(y.T)).real / self.dim

    def encode(self, n: int) -> np.ndarray:
        """
        Encode integer `n` into an RHC vector which is the Hadamard product
        of each the moduli vectors exponentiated to `n`.

        Returns:
            A complex `self.dim`-dimensional vector.
        """
        prod = np.ones(self.dim)
        for mod in self._phis.values():
            prod = self.bind(prod, mod)
        return prod**n

    def _residue_decode(self, es: List[int]) -> int:
        """
        Decode a list of integers using the Chinese remainder theorem to its
        corresponding integer.

        For more information about the algorithm used, refer to Garner (1959),
        and https://personal.utdallas.edu/~ivor/ce6305/m5p.pdf.

        Returns:
            An integer corresponding to the unique list of values provided by
                `es`.
        """
        M = int(np.prod(self.moduli))
        x = 0
        for i in range(len(self.moduli)):
            m_i = self.moduli[i]
            M_I = M // m_i
            a_i = pow(M_I, -1, mod=m_i)
            x += M_I * ((a_i * es[i]) % m_i)

        return x % M

    def gt(self, x: np.ndarray, y: np.ndarray) -> bool:
        raise NotImplementedError("TODO")

    def lt(self, x: np.ndarray, y: np.ndarray) -> bool:
        raise NotImplementedError("TODO")

    def _activation(self, x: np.ndarray) -> np.ndarray:
        """Activation function used in `decode`."""
        f = np.vectorize(lambda theta: theta + np.sin(theta))
        return np.exp(cmath.sqrt(-1) * f(np.abs(x)))


    def decode(self, x: np.ndarray, max_iterations: int = 20) -> int:
        """
        Decode some `RHC` hypervector `x` into an integer using a
        resonator network.
        Returns:
            An integer.
        """
        factors = []
        for modulus in self.moduli:
            factors.append(np.sum(self._codebook[modulus], axis=1))

        runs = [factors]
        for i in range(max_iterations):
            factors = runs[i]
            update_factors = []
            for j, factor in enumerate(factors):
                codebook = self._codebook[self.moduli[j]]
                other_factors = [
                    other_factor
                    for other_factor in factors
                    if not np.array_equal(other_factor, factor)
                ]

                bins = np.ones_like(x)
                for k, other_factor in enumerate(other_factors):
                    bins = self.bind(bins, other_factor)

                x_0 = self.bind(x, bins)
                cleanup = np.dot(codebook, codebook.T)
                res = self._activation(np.dot(cleanup, x_0))
                update_factors.append(res)
            runs.append(update_factors)
        return runs

    def vector_gen(self) -> np.ndarray:
        raise TypeError(
            "TypeError: cannot just generate new RHC vectors in this implementation"
        )

    def __getitem__(self, key: str) -> np.ndarray:
        return self.symbols[key]

In [275]:
rhc = RHC(dim=1000, moduli=[3, 5, 7, 11])

x2 = rhc.encode(2)
decoded = rhc.decode(x2, max_iterations=30)

for j in range(len(decoded)):
    hat_z1 = decoded[j]
    for i, vector in enumerate(rhc._codebook[3].T):
        print(f"similarity between z_3({i+1}) and est z_3")
        print(rhc.sim(vector, hat_z1[0]))
    print()

similarity between z_3(1) and est z_3
0.9870000000000002
similarity between z_3(2) and est z_3
0.9869999999999998
similarity between z_3(3) and est z_3
0.987

similarity between z_3(1) and est z_3
0.14408490013428368
similarity between z_3(2) and est z_3
0.15137225262819543
similarity between z_3(3) and est z_3
0.6915428472375229

similarity between z_3(1) and est z_3
-0.1577405413693823
similarity between z_3(2) and est z_3
-0.1577405413693823
similarity between z_3(3) and est z_3
0.8487594586306191

similarity between z_3(1) and est z_3
0.3127085041202072
similarity between z_3(2) and est z_3
-0.17262917197429745
similarity between z_3(3) and est z_3
0.8469206678540919

similarity between z_3(1) and est z_3
0.16146796275082517
similarity between z_3(2) and est z_3
-0.32386971334367937
similarity between z_3(3) and est z_3
0.6956801264847097

similarity between z_3(1) and est z_3
-0.006500000000000055
similarity between z_3(2) and est z_3
-0.006499999999999737
similarity between z_3(3

# Calculator